In [ ]:
import json
import subprocess
import os

import altair as alt
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
# alt.data_transformers.enable('csv')
alt.data_transformers.disable_max_rows()

In [ ]:
ips = [
    "34.121.51.252",
    "34.27.25.196",
    "35.239.96.48",
    "34.70.74.250",
    "34.170.237.183",
    "34.69.129.190",
    "35.223.243.136",
]

In [ ]:
# for ip in tqdm(ips):
#     process = subprocess.Popen(f'scp -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null chanwutk@{ip}:"/home/chanwutk/code/apperception/outputs/*" ../outputs/', shell=True)
#     process.wait()

In [ ]:
DATA_DIR = '../outputs'

In [ ]:
failed_videos = {}
for f in os.listdir(DATA_DIR):
    if not f.startswith('failed_videos'): continue

    with open(os.path.join(DATA_DIR, f), 'r') as _f:
        content = json.load(_f)

    name = f[len('failed_videos--'):-len('.json')]
    failed_videos[name] = []
    for v_name, err in content:
        failed_videos[name].append(v_name)

In [ ]:
stage_order = [
    'InView', 'DecodeFrame', 'Detection2D.YoloDetection', 'Detection2D.ObjectTypeFilter',
    'DepthEstimation', 'Detection3D.FromDetection2DAndDepth', 'Detection3D.FromDetection2DAndRoad',
    'DetectionEstimation', 'Tracking2D.StrongSORT', 'Tracking3D.FromTracking2DAndDepth',
    'Tracking3D.FromTracking2DAndRoad', 'SegmentTrajectory.FromTracking3D'
]

test_names = {
    'de': 'Only Detection Estimation',
    'noopt': 'Baseline',
    'inview': 'Only In-View',
    'geo': 'Only Geo Depth Estimation',
    'objectfilter': 'Only Object Filter',
    'opt': 'Optimized',
    'optde': 'Optimized with Detection Estimation'
}

test_order = [
    'noopt',
    'inview',
    'objectfilter',
    'geo',
    'de',
    'opt',
    'optde'
]

In [ ]:
def combine_perf():
    data = []
    for filename in os.listdir(DATA_DIR):
        if filename.startswith('perf'):
            test, run = filename.split("--")[1].split(".")[0].split("_")
            with open(os.path.join(DATA_DIR, filename), 'r') as f:
                for stage in json.load(f):
                    stagename = stage['stage']
                    for b in stage['benchmark']:
                        after, before = b['keep']
                        assert after <= before
                        skip = (1. - (after / before)) if before != 0 else 0
                        data.append({
                            **b,
                            'stage': stagename,
                            '_test': test,
                            'test': test_names[test],
                            'test_order': test_order.index(test),
                            'run': run,
                            'name': b['name'].split('/')[-1].split('.')[0],
                            'skip': skip,
                            'stage_order': stage_order.index(stagename)
                        })
    return data


perf = combine_perf()

In [ ]:
# with open('./output/perf.json', 'w') as f:
#     json.dump([p for p in perf if p['run'] == "0"], f, indent=1)

In [ ]:
len([p for p in perf if p['run'] == "0"])

In [ ]:
perf[0]

In [ ]:
failed_videos

In [ ]:
df_perf = pd.DataFrame.from_dict(perf)
# df_perf = df_perf[df_perf['run'] == "0"]
df_perf[:20]

In [ ]:
df_perf.groupby(['test', 'stage']).count()

In [ ]:
(alt.Chart(df_perf)
    .mark_bar()
    .encode(
        x='average(runtime)',
        y=alt.Y('test:O', sort=alt.Sort({'field': 'test_order'})),
        color=alt.Color('stage:N', sort=stage_order),
        order=alt.Order('order:O')
    )
    .properties(height=300, width=800)
)

In [ ]:
df = df_perf[(df_perf.stage == 'DetectionEstimation') | (df_perf.stage == 'Tracking2D.StrongSORT')]
df_noopt = df[df._test == 'noopt']
df_de = df[df._test == 'de']
df_opt = df[df._test == 'opt']
df_optde = df[df._test == 'optde']

In [ ]:
len(df_noopt), len(df_de[df_de.run == '0'])

In [ ]:
df_baseline = (df_noopt[df_noopt.stage == 'Tracking2D.StrongSORT'].set_index(['name', 'run'])
    .join(df_de[df_de.stage == 'Tracking2D.StrongSORT'].set_index(['name', 'run']), lsuffix="_noOpt", rsuffix="_de", how='inner')
    # .set_index(['name', 'run'])
    .join(df_de[df_de.stage == 'DetectionEstimation'].set_index(['name', 'run']), lsuffix="_SS", rsuffix="_DE", how='inner')
)[['runtime_noOpt', 'keep_noOpt', 'stage_noOpt', 'runtime_de', 'keep_de', 'stage_de', 'runtime', 'keep', 'stage']]
df_baseline

In [ ]:
def _(x):
    return pd.Series(
        [x.runtime_noOpt, x.keep_noOpt[1], x.runtime, x.runtime_de, x.keep[0], x.run],
        index=['runtime_before', 'frames_before', 'runtime_ss_after', 'runtime_de_after', 'frames_after', 'run']
    )


df_baseline_ = df_baseline.reset_index().apply(_, axis=1)
df_baseline_

In [ ]:
(alt.Chart(
        df_baseline_,
        title="Runtime Reduction vs Frame Skip Rate"
    )
    .mark_point()
    .transform_calculate(
        'runtime_reduction',
        calculate='1 - (datum.runtime_ss_after + datum.runtime_de_after) / datum.runtime_before'
    )
    .transform_calculate(
        'frames_reduction',
        calculate='1 - datum.frames_after / datum.frames_before'
    )
    .transform_filter('datum.run === "0"')
    .encode(
        x=alt.X('frames_reduction:Q', title='Frame Skip Rate'),
        y=alt.Y('runtime_reduction:Q', title='Runtime Reduction'),
    )
)

In [ ]:
(alt.Chart(
        df_baseline_,
        title="Runtime Reduction vs Detection Estimation Runtime"
    )
    .mark_point()
    .transform_calculate(
        'runtime_reduction',
        calculate='1 - (datum.runtime_ss_after + datum.runtime_de_after) / datum.runtime_before'
    )
    .transform_filter('datum.run === "0"')
    .encode(
        x=alt.X('runtime_de_after:Q', title='Detection Estimation Runtime'),
        y=alt.Y('runtime_reduction:Q', title='Runtime Reduction'),
    )
)

In [ ]:
(alt.Chart(
        df_baseline_,
        title="Frame Skip Rate vs Detection Estimation Runtime"
    )
    .mark_point()
    .transform_calculate(
        'frames_reduction',
        calculate='1 - datum.frames_after / datum.frames_before'
    )
    .transform_filter('datum.run === "0"')
    .encode(
        x=alt.X('runtime_de_after:Q', title='Detection Estimation Runtime'),
        y=alt.Y('frames_reduction:Q', title='Frame Skip Rate'),
    )
)

In [ ]:
alt.Chart(
    df_baseline_,
    title='Cumulative Frequency Distribution of Detection Estimation\'s skip ratio'
).transform_calculate(
    'skip_ratio',
    calculate='1 - (datum.frames_after / datum.frames_before)'
).transform_window(
    ECDF="cume_dist()",
    sort=[{"field": "skip_ratio"}],
).mark_line().encode(
    x=alt.X("skip_ratio:Q", title='Frame Skip Ratio'),
    y="ECDF:Q"
).properties(width=700)

In [ ]:
alt.Chart(
    df_baseline_,
    title='Cumulative Frequency Distribution of Runtime Reduction (%)'
).transform_calculate(
    'runtime_reduction',
    calculate='100*(1 - ((datum.runtime_ss_after + datum.runtime_de_after) / datum.runtime_before))'
).transform_window(
    ECDF="cume_dist()",
    sort=[{"field": "runtime_reduction"}],
).mark_line().encode(
    x=alt.X("runtime_reduction:Q", title='Runtime Reduction (%)'),
    y="ECDF:Q"
).properties(width=700)